# Custom Square Sudoku

For all Square Sudoku: 16 x 16, 25 x 25, 36 x 36 ... 


In [ ]:
from ortools.sat.python import cp_model as cp
from Utils.common import get_exist
import math 
import os 

def read_instance_sudoku_custom_size(path, is_problem = True):
    
    if is_problem:
        file_name = f"./assets/Sudoku/16x16Sudoku/problems/{path}"
    else:
        file_name = f"./assets/Sudoku/16x16Sudoku/solutions/{path}"

    m = 0
    n = 0
    grids = None
    print( f"READING FILE {file_name}" )
    if not os.path.isfile(file_name):
        print('File does not exist')
        return None, None, None
    else:

        with open(file_name) as file:
            firstline = file.readline().strip().split(" ")
            m = int(firstline[0])
            n = int(firstline[1])
            grids = [[0 for _ in range(n)] for _ in range(m)]

            for idx in range(m):
                line = file.readline().strip().split(" ")
                assert len(line) == n
                
                for j in range(n):
                    if line[j] not in [ '.', '-']:
                        grids[idx][j] = line[j]
                    else:
                        grids[idx][j] = '-'
        return m, n, grids

def check_sudoku_custom_size(path, solver_result):
    m, n, grid = read_instance_sudoku_custom_size(path, False)
    if not grid or not m or not n or not isinstance(m, int) or not isinstance(n, int):
        print(f"Not found {path} in folder.")
        return False
    
    if not solver_result or not isinstance(solver_result, list):
        print(f"Invalid solver result, instance {path}")
        return False

    if len(solver_result) != m or len(solver_result[0]) != n:
        print(f"Invalid solver result size, instance {path}")
        return False
    
    valid_nums = set(str(k) for k in range(1, m + 1))
    
    for i in range(m):
        for j in range(n):
            if grid[i][j] not in valid_nums:
                print(f"Invalid answer grid. {grid[i][j]}, {path}")
                return False
            
    board_result = list(map(lambda sublist: list(map(int, sublist)), grid))
    for i in range(m):
        for j in range(n):
            if board_result[i][j] != solver_result[i][j]:
                print(f"WRONG ANSWER! {i},{j}. {path}")
                return False
    print(f"Correct. {path}")
    return True

class sudoku_custom_size:
    
    def __init__(self, grid, size):
        if not isinstance(grid, list):
            print("Check grid type.")
            return None

        if not isinstance(size, int) or int(math.sqrt(size)) ** 2 != size:
            print("Check size.")
            return None
        
        if len(grid) != size or len(grid[0]) != size:
            print(f"Check grid type. Not {size} x {size} grid.")
            return None
        
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        self.size = size
        self.cell_size = int(math.sqrt(size))
        self.grid = [element for row in grid for element in row]
        self.x = {}
        self.result = None
        
        valid_nums = set(str(k) for k in range(1, size + 1))
        for i in range(size):
            for j in range(size):
                if self.grid[size * i + j] in ["0", "-"]:
                    self.x[i, j] = self.model.NewIntVar(1, size, f'x[{i}, {j}]')
                elif self.grid[size * i + j] in valid_nums:
                    self.x[i, j] = int(self.grid[size * i + j])
                else:
                    print("Contain invalid nums.")
                    return None 
    
    def addStdConstr(self):
        for i in range(self.size):
            row = [self.x[i, j] for j in range(self.size)]
            self.model.AddAllDifferent(row)
            col = [self.x[j, i] for j in range(self.size)]
            self.model.AddAllDifferent(col)
        
        for i in range(self.cell_size - 1):
            for j in range(self.cell_size - 1):
                l = self.cell_size
                cell = [
                    self.x[r, c]
                    for r in range(i* l, i * l + l)
                    for c in range(j* l, j * l + l)
                ]
                self.model.AddAllDifferent(cell)
    
    def forgeGrid(self, print_grid = True):    
        
        """_summary_
            Print the final grid
        """
        self.result = []
        
        for i in range(self.size):
            row = [self.solver.Value(self.x[i, j]) for j in range(self.size)]
            cell_width = len(str(self.size)) + 1
            if print_grid:
                print(" ".join(f"{str(num):>{cell_width}}" for num in row))
            self.result.append(row) 
        
        if print_grid:
            print("NumConflicts:", self.solver.NumConflicts())
            print("NumBranches:", self.solver.NumBranches())
            print("WallTime:", self.solver.WallTime())
        
    def solveall(self):
        self.addStdConstr()
        status = self.solver.Solve(self.model)
        
        if status == cp.OPTIMAL:
            self.forgeGrid(print_grid = False)
            return self.result

        elif status == cp.INFEASIBLE:
            print("Infeasible model.")
            return []
            # self.model.ExportToFile("./help.txt")
        else:
            print("Exceed Maximum Time Limit.")
            return []

if __name__ == "__main__":
    
    instance_list = get_exist("./assets/Sudoku/16x16Sudoku/problems")
    # print(instance_list)
    for instance_name in instance_list:
        # instance_name = "301_16x16.txt"
        m, n, grid = read_instance_sudoku_custom_size(instance_name)
        custom_sudoku = sudoku_custom_size(grid, m)
        res = custom_sudoku.solveall()
        check_sudoku_custom_size(instance_name, res)

# Sudoku Variants with multiple standard $9 \times 9$ grids

Which contains: 'samurai', 'sohei', 'sumo', 'windmill', 'shogun', 'gattai8', 'butterfly'

In [ ]:
from ortools.sat.python import cp_model as cp
from Utils.common import get_exist
import math 
import os 

def read_instance_sudoku_variants(path, category, is_problem = True):
    
    if category not in ['samurai', 'sohei', 'sumo', 'windmill', 'shogun', 'gattai8', 'butterfly']:
        print(f"Check Type. {category}")
        return None, None, None

    if is_problem:
        file_name = f"./assets/Sudoku/{category.capitalize()}Sudoku/problems/{path}"
    else:
        file_name = f"./assets/Sudoku/{category.capitalize()}Sudoku/solutions/{path}"

    m = 0
    n = 0
    grids = None
    print( f"READING FILE {file_name}" )
    if not os.path.isfile(file_name):
        print('File {file_name} does not exist')
        return None, None, None
    else:

        with open(file_name) as file:
            firstline = file.readline().strip().split(" ")
            m = int(firstline[0])
            n = int(firstline[1])
            grids = [["-" for _ in range(n)] for _ in range(m)]

            for idx in range(m):
                line = file.readline().strip().split(" ")
                assert len(line) == n
                
                for j in range(n):
                    if line[j] not in [ '.', '-']:
                        grids[idx][j] = line[j]
                    else:
                        grids[idx][j] = '-'
        return m, n, grids

def check_sudoku(path, category, solver_result):
    m, n, grid = read_instance_sudoku_variants(path, category, False)
    if not grid or not m or not n or not isinstance(m, int) or not isinstance(n, int):
        print(f"Not found {path} in folder.")
        return False
    
    if not solver_result or not isinstance(solver_result, list):
        print(f"Invalid solver result, instance {path}")
        return False

    if len(solver_result) != m or len(solver_result[0]) != n:
        print(f"Invalid solver result size, instance {path}")
        return False
    
    valid_nums = set(str(k) for k in range(1, 10))
    
    for i in range(m):
        for j in range(n):
            if str(grid[i][j]) != str(solver_result[i][j]):
                print(f"WRONG ANSWER! {i},{j}. {path}; {grid[i][j]}, {solver_result[i][j]}")
                return False

    print(f"Correct. {path}")
    return True


class sudoku_variants:
    
    def __init__(self, grid, category):
        if not isinstance(grid, list):
            print(f"Check grid type.")
            return None

        if not isinstance(category, str):
            print(f"Check category type {category}.")
            return None
        
        avail_cat = ['samurai', 'sohei', 'sumo', 'windmill', 'shogun', 'gattai8', 'butterfly']
        # Update here!
        
        if category not in avail_cat:
            print(f"Check category {category}. Not exist in {avail_cat}")
            return None
        
        if category == "samurai":
            height = 21
            width = 21
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 0], [0, 12], [12, 0], [6, 6], [12, 12]]
                
        
        elif category == "sohei":
            height = 21
            width = 21
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 6], [6, 0], [6, 12], [12, 6]]
        
        elif category == "sumo":
            height = 33
            width = 33
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 0], [0, 12], [0, 24], 
                               [6, 6], [6, 18], 
                               [12, 0], [12, 12], [12, 24], 
                               [18, 6], [18, 18], 
                               [24, 0], [24, 12], [24,24]]

        elif category == "windmill":
            height = 21
            width = 21
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 3], [9, 0], [3, 12], [6, 6], [12, 9]]
        
        elif category == "shogun":
            height = 21
            width = 45
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 0], [0, 12], [0, 24], [0, 36], 
                               [6, 6], [6, 18], [6, 30],
                               [12, 0], [12, 12], [12, 24], [12, 36]]
        
        elif category == "gattai8":
            height = 21
            width = 33
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 0], [0, 12], [0, 24], 
                               [6, 6], [6, 18],
                               [12, 0], [12, 12], [12, 24]]
        
        elif category == "butterfly":
            height = 12
            width = 12
            if len(grid) != height or not isinstance(grid[0], list) or len(grid[0]) != width:
                print(f"Check {category} grid type. Not {height} x {width} grid.")
                return 
            else:
                self.height = height
                self.width = width
                self.pivots = [[0, 0], [0, 3], [3, 0], [3, 3]]
        
        self.category = category
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        self.grid = grid
        self.x = {}
        self.result = None
        
        
        valid_nums = set([str(k) for k in range(1, 10)] + ['-', '0'])
        for i in range(self.height):
            for j in range(self.width):
                if self.grid[i][j] not in valid_nums:
                    print("Contain invalid nums.")
                    return None 
                else:
                    if self.isCandidatePos(i, j):
                        if self.grid[i][j] in ["-", "0"]:
                            self.x[i, j] = self.model.NewIntVar(1, 9, f'x[{i}, {j}]')
                        else:
                            self.x[i, j] = int(self.grid[i][j])
                    else:
                        continue

    def addStdConstrByNine(self, a, b):
        for i in range(9):
            row = [self.x[a + i, b + j] for j in range(9)]
            self.model.AddAllDifferent(row)
            col = [self.x[a + j, b + i] for j in range(9)]
            self.model.AddAllDifferent(col)
        
        for i in range(3):
            for j in range(3):
                l = 3
                cell = [
                    self.x[r + a, c + b]
                    for r in range(i * l, i * l + l)
                    for c in range(j * l, j * l + l)
                ]
                self.model.AddAllDifferent(cell)
    
    def isCandidatePos(self, a, b):
        for pivot in self.pivots:
            if pivot[0] <= a < pivot[0] + 9 and pivot[1] <= b < pivot[1] + 9:
                return True 
        return False
    
    def forgeGrid(self, print_grid = False):    
        
        """_summary_
            Print the final grid
        """
        self.result = []
        
        # if self.category in ["samurai", "sohei"]:
        for i in range(self.height):
            row = [self.solver.Value(self.x[i, j])  if self.isCandidatePos(i, j) else "-" for j in range(self.width)]
            cell_width = 2
            if print_grid:
                print(" ".join(f"{str(num):>{cell_width}}" for num in row))
            self.result.append(row) 
                
        
        if print_grid:
            print("NumConflicts:", self.solver.NumConflicts())
            print("NumBranches:", self.solver.NumBranches())
            print("WallTime:", self.solver.WallTime())
        
    def solveall(self):
        for pivot in self.pivots:
            self.addStdConstrByNine(pivot[0], pivot[1])

        status = self.solver.Solve(self.model)
        
        if status == cp.OPTIMAL:
            self.forgeGrid(print_grid = False)
            return self.result

        elif status == cp.INFEASIBLE:
            print("Infeasible model.")
            return []
            # self.model.ExportToFile("./help.txt")
        else:
            print("Exceed Maximum Time Limit.")
            return []

if __name__ == "__main__":
    
    category = "butterfly"
    
    # Every category in ['samurai', 'sohei', 'sumo', 'windmill', 'shogun', 'gattai8', 'butterfly']
    # has been checked okay.
    instance_list = get_exist(f"./assets/Sudoku/{category.capitalize()}Sudoku/problems")
    for instance_name in instance_list:
        # instance_name = "301_16x16.txt"
        print(instance_name)
        m, n, grid = read_instance_sudoku_variants(instance_name, category)
        sudoku_variant = sudoku_variants(grid, category)
        if sudoku_variant:
            res = sudoku_variant.solveall()
            print(res)
            check_sudoku(instance_name, category, res)
        else:
            print("No avail sudoku grid!")

# Standard $9 \times 9$ with different rules

In [ ]:

from itertools import combinations
from ortools.sat.python import cp_model as cp
import math

class CompoundSudokuSolver:
    
    def __init__(
        self, 
        grid, 
        X = 9, 
        Y = 9, 
        std_rule = True,
        killer = None,
        petite_killer = None,
        consecutive = None,
        sandwich = None,
        anti_knight = None,
        anti_king = None,
        arrow = None,
        thermo = None, 
        greater_than = None,
        jigsaw = None,
        vudoku = None,
        all_nine = None,
        XV = None,
        even_odd = None,
        kropki = None,
        diagonal = False) -> None:
        
        self.grid = grid
        self.X = X
        self.Y = Y
        self.std_rule = std_rule
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        self.x = {}
        self.killer = killer
        self.petite_killer = petite_killer
        self.consecutive = consecutive
        self.sandwich = sandwich
        self.anti_knight = anti_knight
        self.anti_king = anti_king
        self.arrow = arrow
        self.thermo = thermo
        self.diagonal = diagonal
        self.greater_than = greater_than
        self.jigsaw = jigsaw
        self.vudoku = vudoku
        self.all_nine = all_nine
        self.XV = XV
        self.evan_odd = even_odd
        self.kropki = kropki
        # list (tuple), 
        # 7 2
        # 8 2
        # 10 3 
        # first tuple value indicates the index of corner
        # second tuple value indicates the shape of corner:
        #     0:    ｜        2:--｜
        #         __｜            ｜
        # 
        #     1:  ｜          3: ｜--
        #         ｜__           ｜

        self.aux_sand = None
        self.vudoku_aux = None
        self.consecutive_aux = None
        self.result = ""
        
        assert X == Y

        for i in range(Y):
            for j in range(X):
                if self.grid[i * self.Y + j] == "0":
                    self.x[i, j] = self.model.NewIntVar(1, self.X, f'x[{i}, {j}]')
                else:
                    self.x[i, j] = int(self.grid[i * self.X + j])
                
    def addStandardConstr(self):
        
        """_summary_
            Add standard Sudoku Constraint
        """
        
        for i in range(self.Y):
            row = [self.x[i, j] for j in range(self.X)]
            self.model.AddAllDifferent(row)
            col = [self.x[j, i] for j in range(self.Y)]
            self.model.AddAllDifferent(col)
        
        for i in range(int(math.sqrt(self.X)) - 1):
            for j in range(int(math.sqrt(self.Y)) - 1):
                l = int(math.sqrt(self.X))
                cell = [
                    self.x[r, c]
                    for r in range(i* l, i * l + l)
                    for c in range(j* l, j * l + l)
                ]
                self.model.AddAllDifferent(cell)
    
    
    def addConsecutiveConstr(self):
    # c = sum([x[i] == val for i in range(len(x))])
    
    # n = len(x)
    # b = [model.NewBoolVar(f"b[{i}]")  for i in range(n)]
    # for i in range(n):
    #     model.Add((x[i] == val)).OnlyEnforceIf(b[i])
    #     model.Add((x[i] != val)).OnlyEnforceIf(b[i].Not())
    # model.Add(c == sum(b))
        self.consecutive_aux = {}
        for idx, sub_char in enumerate(self.consecutive):
            sub_a, sub_b = idx // 17, idx % 17
            if sub_char == ".":
                self.consecutive_aux[f"{sub_a}_{sub_b}_1"] = self.model.NewBoolVar(name = f"{sub_a}_{sub_b}_1")
                self.consecutive_aux[f"{sub_a}_{sub_b}_2"] = self.model.NewBoolVar(name = f"{sub_a}_{sub_b}_2")
                if sub_b <= 7:
                    self.model.Add(self.x[int(sub_a), int(sub_b)] - self.x[int(sub_a), int(sub_b) + 1] == 1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_1"])
                    self.model.Add(self.x[int(sub_a), int(sub_b)] - self.x[int(sub_a), int(sub_b) + 1] != 1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_1"].Not())
                    self.model.Add(self.x[int(sub_a), int(sub_b)] - self.x[int(sub_a), int(sub_b) + 1] == -1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_2"])
                    self.model.Add(self.x[int(sub_a), int(sub_b)] - self.x[int(sub_a), int(sub_b) + 1] != -1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_2"].Not())
                    self.model.Add(self.consecutive_aux[f"{sub_a}_{sub_b}_1"] + self.consecutive_aux[f"{sub_a}_{sub_b}_2"] == 0)
                elif sub_b <= 16:
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] - self.x[int(sub_a) + 1, int(sub_b) - 8] == 1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_1"])
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] - self.x[int(sub_a) + 1, int(sub_b) - 8] != 1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_1"].Not())
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] - self.x[int(sub_a) + 1, int(sub_b) - 8] == -1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_2"])
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] - self.x[int(sub_a) + 1, int(sub_b) - 8] != -1).OnlyEnforceIf(self.consecutive_aux[f"{sub_a}_{sub_b}_2"].Not())
                    self.model.Add(self.consecutive_aux[f"{sub_a}_{sub_b}_1"] + self.consecutive_aux[f"{sub_a}_{sub_b}_2"] == 0)
                continue
            if sub_b <= 7:
                self.model.AddAbsEquality(1, self.x[int(sub_a), int(sub_b)] - self.x[int(sub_a), int(sub_b) + 1])
            elif sub_b <= 16:
                self.model.AddAbsEquality(1, self.x[int(sub_a), int(sub_b) - 8] - self.x[int(sub_a) + 1, int(sub_b) - 8])
        # for _, cage in enumerate(self.consecutive):
        #     self.model.AddAbsEquality(1, self.x[cage[0][0], cage[0][1]] - self.x[cage[1][0], cage[1][1]])
             
             
    def addDiagonalConstr(self):
        
        """_summary_
            Add diagonal Constraints.
        """
        
        ltr = []
        rtl = []
        for i in range(self.Y):
            ltr.append(self.x[i, i])
            rtl.append(self.x[self.X - i - 1, i])
        self.model.AddAllDifferent(ltr)
        self.model.AddAllDifferent(rtl)
            
    def printgrid(self):    
        
        """_summary_
            Print the final grid
        """
    
        for i in range(self.Y):
            for j in range(self.X):
                print(self.solver.Value(self.x[i, j]), end=" ")
                self.result += str(self.solver.Value(self.x[i, j]))
            print()
        print()

        print("NumConflicts:", self.solver.NumConflicts())
        print("NumBranches:", self.solver.NumBranches())
        print("WallTime:", self.solver.WallTime())
        
        # for i in range(9):
        #     for j in range(6):
        #         print(self.solver.Value(self.aux_sand["R", i, j]))
    
    def addKillerConstr(self):
        """_summary_
            Add killer constraint from data
        """
        
        for (res, segment) in self.killer:
            cage = [self.x[i[0], i[1]] for i in segment]
            self.model.Add(sum(cage) == res)
            self.model.AddAllDifferent(cage)
    
    def addPetiteKillerConstr(self):
        """_summary_
            增加小杀手数独约束
        Raises:
            Exception: _description_
            Exception: _description_
        """
        if len(self.petite_killer[0]) != self.X:
            raise Exception("小杀手数独斜行和数量有误。")
        for line in self.petite_killer:
            if line[0] == "TL":
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(i + 1):
                        constrCages.append(self.x[j, i - j])
                    self.model.Add(sum(constrCages) == line[i + 1])

            elif line[0] == "RT":
                
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(i + 1):
                        constrCages.append(self.x[j, self.X - i + j - 1])
                    self.model.Add(sum(constrCages) == line[i + 1])

            elif line[0] == "BR":
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(self.X - 1 - i):
                        constrCages.append(self.x[self.X - 1 - j, 1 + i +  j])
                    self.model.Add(sum(constrCages) == line[i + 1])

                    
            elif line[0] == "LB":
            
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(self.X - 1 - i):
                        constrCages.append(self.x[1 + j + i,  j])
                    self.model.Add(sum(constrCages) == line[i + 1])

            else:
                raise Exception("检查小杀手数独位置元素字符串, 必须为TL / RT / BR / LB 中的一个")

    def addThermoConstr(self):
        """_summary_
            Thermo constraints
        """
        for thermo_ in self.thermo:
            for idx in range(len(thermo_) - 1):
                self.model.Add(self.x[thermo_[idx][0], thermo_[idx][1]] < self.x[thermo_[idx + 1][0], thermo_[idx + 1][1]])
        
    def addSandwichConstr(self):
        """
            Add sandwich Sudoku Constrs
        """
        self.aux_sand = {}
        
        # x[i,j] = 9 -> Bool = True
        # x[i,j] <= 8 -> Bool = False
        # False -> x[i, j] <= 8
        
        # True
        for nums in self.sandwich:
            if nums[0] == "R":
                for idx, num in enumerate(nums[1:]):
                    if num < 0:
                        continue
                    # cnt = 0
                    for index, cbn in enumerate(combinations([i for i in range(self.X)], 2)):

                        self.aux_sand["R", idx, index,  0] = self.model.NewBoolVar(f"R, {idx}, {index}, 0")
                        self.aux_sand["R", idx, index,  1] = self.model.NewBoolVar(f"R, {idx}, {index}, 1")
                        self.aux_sand["R", idx, index,  2] = self.model.NewBoolVar(f"R, {idx}, {index}, 2")
                        self.aux_sand["R", idx, index,  3] = self.model.NewBoolVar(f"R, {idx}, {index}, 3")
                        self.aux_sand["R", idx, index,  4] = self.model.NewBoolVar(f"R, {idx}, {index}, 4")
                        self.aux_sand["R", idx, index,  5] = self.model.NewBoolVar(f"R, {idx}, {index}, 5")

                        
                        self.model.Add(self.x[cbn[0], idx] == 1).OnlyEnforceIf(self.aux_sand["R", idx, index, 0])
                        self.model.Add(self.x[cbn[0], idx] >= 2).OnlyEnforceIf(self.aux_sand["R", idx, index, 0].Not())

                        
                        self.model.Add(self.x[cbn[1], idx] == 9).OnlyEnforceIf(self.aux_sand["R", idx, index, 1])
                        self.model.Add(self.x[cbn[1], idx] <= 8).OnlyEnforceIf(self.aux_sand["R", idx, index, 1].Not())
                        
                        self.model.Add(self.aux_sand["R", idx,  index, 0] + self.aux_sand["R", idx, index,  1] == 2).OnlyEnforceIf(self.aux_sand["R", idx, index, 4])
                        self.model.Add(self.aux_sand["R", idx,  index, 0] + self.aux_sand["R", idx, index,  1] <= 1).OnlyEnforceIf(self.aux_sand["R", idx, index, 4].Not())
                        
                        self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                        # self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] - 1)) != num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                        
                        self.model.Add(self.x[cbn[0], idx] == 9).OnlyEnforceIf(self.aux_sand["R", idx, index,  2])
                        self.model.Add(self.x[cbn[0], idx] <= 8).OnlyEnforceIf(self.aux_sand["R", idx, index,  2].Not())

                        
                        self.model.Add(self.x[cbn[1], idx] == 1).OnlyEnforceIf(self.aux_sand["R", idx, index,  3])
                        self.model.Add(self.x[cbn[1], idx] >= 2).OnlyEnforceIf(self.aux_sand["R", idx, index,  3].Not())

                        
                        self.model.Add(self.aux_sand["R", idx, index, 2] + self.aux_sand["R", idx, index,  3] == 2).OnlyEnforceIf(self.aux_sand["R", idx, index, 5])
                        self.model.Add(self.aux_sand["R", idx, index, 2] + self.aux_sand["R", idx, index,  3] <= 1).OnlyEnforceIf(self.aux_sand["R", idx, index, 5].Not())
                        
                        self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 5])
                       
                   
                        
            elif nums[0] == "C":
                if nums[0] == "R":
                    for idx, num in enumerate(nums[1:]):
                        if num < 0:
                            continue
                        # cnt = 0
                        for index, cbn in enumerate(combinations([i for i in range(self.X)], 2)):
                            self.aux_sand["C", idx, index,  0] = self.model.NewBoolVar(f"R, {idx}, {index}, 0")
                            self.aux_sand["C", idx, index,  1] = self.model.NewBoolVar(f"R, {idx}, {index}, 1")
                            self.aux_sand["C", idx, index,  2] = self.model.NewBoolVar(f"R, {idx}, {index}, 2")
                            self.aux_sand["C", idx, index,  3] = self.model.NewBoolVar(f"R, {idx}, {index}, 3")
                            self.aux_sand["C", idx, index,  4] = self.model.NewBoolVar(f"R, {idx}, {index}, 4")
                            self.aux_sand["C", idx, index,  5] = self.model.NewBoolVar(f"R, {idx}, {index}, 5")

                            
                            self.model.Add(self.x[idx, cbn[0]] == 1).OnlyEnforceIf(self.aux_sand["C", idx, index, 0])
                            self.model.Add(self.x[idx, cbn[0]] >= 2).OnlyEnforceIf(self.aux_sand["C", idx, index, 0].Not())

                            
                            self.model.Add(self.x[idx, cbn[1]] == 9).OnlyEnforceIf(self.aux_sand["C", idx, index, 1])
                            self.model.Add(self.x[idx, cbn[1]] <= 8).OnlyEnforceIf(self.aux_sand["C", idx, index, 1].Not())
                            
                            self.model.Add(self.aux_sand["C", idx,  index, 0] + self.aux_sand["C", idx, index,  1] == 2).OnlyEnforceIf(self.aux_sand["C", idx, index, 4])
                            self.model.Add(self.aux_sand["C", idx,  index, 0] + self.aux_sand["C", idx, index,  1] <= 1).OnlyEnforceIf(self.aux_sand["C", idx, index, 4].Not())
                            
                            self.model.Add(sum(self.x[idx, j] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                            # self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] - 1)) != num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                            
                            self.model.Add(self.x[idx, cbn[0]] == 9).OnlyEnforceIf(self.aux_sand["C", idx, index,  2])
                            self.model.Add(self.x[idx, cbn[0]] <= 8).OnlyEnforceIf(self.aux_sand["C", idx, index,  2].Not())

                            
                            self.model.Add(self.x[idx, cbn[1]] == 1).OnlyEnforceIf(self.aux_sand["C", idx, index,  3])
                            self.model.Add(self.x[idx, cbn[1]] >= 2).OnlyEnforceIf(self.aux_sand["C", idx, index,  3].Not())

                            
                            self.model.Add(self.aux_sand["C", idx, index, 2] + self.aux_sand["C", idx, index,  3] == 2).OnlyEnforceIf(self.aux_sand["C", idx, index, 5])
                            self.model.Add(self.aux_sand["C", idx, index, 2] + self.aux_sand["C", idx, index,  3] <= 1).OnlyEnforceIf(self.aux_sand["C", idx, index, 5].Not())
                            self.model.Add(sum(self.x[idx, j] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4]) 

    def addAntiKnightConstr(self):
        
        offsets = [
            (-2, 1), 
            (-2, -1), 
            (-1, 2), 
            (-1, -2), 
            (1, 2), 
            (1, -2), 
            (2, 1), 
            (2, -1)
        ]
        for i in range(self.Y):
            for j in range(self.X):
                for offset in offsets:
                    if (i + offset[0] >= 0 and i + offset[0] < self.Y) and (j + offset[1] >= 0 and j + offset[1] < self.X):
                        self.model.Add(self.x[i, j] != self.x[i + offset[0], j + offset[1]])
        
    def addAntiKingConstr(self):
        
        offs = [-1, 0, 1]
        for i in range(self.Y):
            for j in range(self.X):
                for ofx in offs:
                    for ofy in offs:
                        if (i + ofx >= 0 and i + ofx < self.Y) and (j + ofy >= 0 and j + ofy < self.X) and (ofy != 0 and ofx != 0):
                            self.model.Add(self.x[i, j] != self.x[i + ofx, j + ofy])
        
    def addArrowConstr(self):

        for arrow_ in self.arrow:
            self.model.Add(self.x[arrow_[0][0], arrow_[0][1]] == sum([self.x[ar[0], ar[1]] for ar in arrow_[1:]]))
            
    def addGreaterThanConstr(self):
        for idx, sub_ in enumerate(self.greater_than):
            if sub_ == ".":
                continue
            sub_a, sub_b = idx // 17, idx % 17
            if sub_ == ">":
                if sub_b <= 7:
                    self.model.Add(self.x[int(sub_a), int(sub_b)] > self.x[int(sub_a), int(sub_b) + 1])
                    print(f"[{int(sub_a)}, {int(sub_b)}] > [{int(sub_a)}, {int(sub_b) + 1}]")
                elif sub_b <= 16:
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] > self.x[int(sub_a) + 1, int(sub_b) - 8])
            elif sub_ == "<":
                if sub_b <= 7:
                    self.model.Add(self.x[int(sub_a), int(sub_b)] < self.x[int(sub_a), int(sub_b) + 1])
                elif sub_b <= 16:
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] < self.x[int(sub_a) + 1, int(sub_b) - 8])

    def addJigsawConstr(self):
        for i in range(self.Y):
            row = [self.x[i, j] for j in range(self.X)]
            self.model.AddAllDifferent(row)
            col = [self.x[j, i] for j in range(self.Y)]
            self.model.AddAllDifferent(col)
        new_grid = dict()
        for idx, char_ in enumerate(self.jigsaw):
            if char_ not in new_grid:
                new_grid[char_] = []
            new_grid[char_].append(idx)
        
        for k, v in new_grid.items():
            cell = [self.x[idx // 9, idx % 9] for idx in v]
            self.model.AddAllDifferent(cell)

    def addVudokuConstr(self):
        self.vudoku_aux = [self.model.NewBoolVar(f"aux_{i}") for i in range(3 * len(self.vudoku))]
        for idx, val  in enumerate(self.vudoku):
            cornerCentral, cornerShape = val[0], val[1]
            xx, yy  = cornerCentral // self.Y, cornerCentral % self.Y
            aroundCorner = []
            if cornerShape == 0:
                aroundCorner = [(xx, yy - 1), (xx - 1, yy)]
            elif cornerShape == 1:
                aroundCorner = [(xx - 1, yy), ((xx, yy + 1))]
            elif cornerShape == 2:
                aroundCorner = [(xx, yy - 1), (xx + 1, yy )]
            else:
                aroundCorner = [(xx, yy + 1), (xx + 1, yy )]

            self.model.Add(
                (self.x[aroundCorner[0][0], aroundCorner[0][1]] + self.x[aroundCorner[1][0], aroundCorner[1][1]] == self.x[xx, yy])).OnlyEnforceIf(self.vudoku_aux[idx * 3])
            self.model.Add(
                (self.x[aroundCorner[0][0], aroundCorner[0][1]] + self.x[aroundCorner[1][0], aroundCorner[1][1]] != self.x[xx, yy])).OnlyEnforceIf(self.vudoku_aux[idx * 3].Not())
            self.model.Add(
                (self.x[aroundCorner[0][0], aroundCorner[0][1]] - self.x[aroundCorner[1][0], aroundCorner[1][1]] == self.x[xx, yy])).OnlyEnforceIf(self.vudoku_aux[idx * 3 + 1])
            self.model.Add(
                (self.x[aroundCorner[0][0], aroundCorner[0][1]] - self.x[aroundCorner[1][0], aroundCorner[1][1]] != self.x[xx, yy])).OnlyEnforceIf(self.vudoku_aux[idx * 3 + 1].Not())
            self.model.Add(
                (self.x[aroundCorner[1][0], aroundCorner[1][1]] - self.x[aroundCorner[0][0], aroundCorner[0][1]] == self.x[xx, yy])).OnlyEnforceIf(self.vudoku_aux[idx * 3 + 2])
            self.model.Add(
                (self.x[aroundCorner[1][0], aroundCorner[1][1]] - self.x[aroundCorner[0][0], aroundCorner[0][1]] != self.x[xx, yy])).OnlyEnforceIf(self.vudoku_aux[idx * 3 + 2].Not())
            self.model.Add(sum(self.vudoku_aux[idx * 3 : idx * 3 + 3]) == 1)
            
            # 限制上述情况只能取一种发生
            
    def addAllNineConstr(self):
        all_nine_dict = dict()
        for idx, cell in enumerate(self.all_nine):
            if cell == "0":
                continue
            else:
                if cell not in all_nine_dict:
                    all_nine_dict[cell] = []
                all_nine_dict[cell].append((idx // self.X, idx % self.Y))
        for _, cells in all_nine_dict.items():
            self.model.AddAllDifferent([self.x[cell[0], cell[1]] for cell in cells])
    
    def addXVConstr(self):
        # Sudoku XV(Evil) https://gridpuzzle.com/vx-sudoku/159j0
        for idx, sub_ in enumerate(self.XV):
            if sub_ == ".":
                continue
            sub_a, sub_b = idx // 17 , idx % 17 
            if sub_ == "V":
                if sub_b <= 7:
                    self.model.Add(self.x[int(sub_a), int(sub_b)] + self.x[int(sub_a), int(sub_b) + 1] == 5)
                elif sub_b <= 16:
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] + self.x[int(sub_a) + 1, int(sub_b) - 8] == 5)
            elif sub_ == "X":
                if sub_b <= 7:
                    self.model.Add(self.x[int(sub_a), int(sub_b)] + self.x[int(sub_a), int(sub_b) + 1] == 10)
                elif sub_b <= 16:
                    self.model.Add(self.x[int(sub_a), int(sub_b) - 8] + self.x[int(sub_a) + 1, int(sub_b) - 8] == 10)
    
    def addEvenOddConstr(self):
        for idx, sub_char in enumerate(self.evan_odd):
            idx_a, idx_b = idx // self.X, idx % self.X
            if sub_char == "E":
                self.model.AddAllowedAssignments([self.x[idx_a, idx_b]], [[2], [4], [6], [8]])
            elif sub_char == "O":
                self.model.AddAllowedAssignments([self.x[idx_a, idx_b]], [[1], [3], [5], [7], [9]])
        
    def addKropkiConstr(self):
        for idx, sub_ in enumerate(self.kropki):
            if sub_ == ".":
                continue
            sub_a, sub_b = idx // 17 , idx % 17 
            if sub_ == "B":
                if sub_b <= 7:
                    self.model.AddAllowedAssignments([self.x[int(sub_a), int(sub_b)], self.x[int(sub_a), int(sub_b) + 1]], [[1,2], [2,4], [3,6], [4,8], [8,4], [6,3], [4,2], [2,1]])
                elif sub_b <= 16:
                    self.model.AddAllowedAssignments([self.x[int(sub_a), int(sub_b) - 8], self.x[int(sub_a) + 1, int(sub_b) - 8]], [[1,2], [2,4], [3,6], [4,8], [8,4], [6,3], [4,2], [2,1]])
            elif sub_ == "W":
                if sub_b <= 7:
                    self.model.AddAbsEquality(1, self.x[int(sub_a), int(sub_b)] - self.x[int(sub_a), int(sub_b) + 1])
                elif sub_b <= 16:
                    self.model.AddAbsEquality(1, self.x[int(sub_a), int(sub_b) - 8] - self.x[int(sub_a) + 1, int(sub_b) - 8])
    
    
    def solveall(self):

        if self.std_rule and self.jigsaw == None:
            self.addStandardConstr()
        
        if self.jigsaw != None:
            self.addJigsawConstr()
        
        if self.diagonal:
            self.addDiagonalConstr()
        if self.killer != None:
            self.addKillerConstr()
        if self.petite_killer != None:
            self.addPetiteKillerConstr()
        if self.consecutive != None:
            self.addConsecutiveConstr()
        if self.sandwich != None:
            self.addSandwichConstr()
        if self.anti_knight != None and self.anti_knight == True:
            self.addAntiKnightConstr()
        if self.anti_king != None and self.anti_king == True:
            self.addAntiKingConstr()
        if self.thermo != None:
            self.addThermoConstr()
        if self.arrow != None:
            self.addArrowConstr()
        if self.greater_than != None:
            self.addGreaterThanConstr()
        if self.vudoku != None:
            self.addVudokuConstr()
        if self.all_nine != None:
            self.addAllNineConstr()
        if self.XV != None:
            self.addXVConstr()
        if self.evan_odd != None:
            self.addEvenOddConstr()
        if self.kropki != None:
            self.addKropkiConstr()
        
        status = self.solver.Solve(self.model)

        if status == cp.OPTIMAL:
            self.printgrid()
            return self.result

        elif status == cp.INFEASIBLE:
            print("模型不可行")
            return ""
            # self.model.ExportToFile("./help.txt")
        else:
            print("无法在规定时间内求解")
            return ""
        # else:
        #     print("没有找到可行方案")
        #     self.model.ExportToFile("./help.txt")

if __name__ == "__main__":
    pass

## Vudoku

- 标准数独规则适用
- 每个 “V”型包含的三个格子，边缘两个格子的数之和或者差，等于中间格子的数字。

-----------

- STD rules applied
- The number at the vertex of each marked "V" must be either the sum or the difference of the other two numbers of the "V".


In [13]:
# 测试Vudoku数独 

if __name__ == "__main__":
    test_vudoku_solver = "960008003000000000000057000100004900000000000000001000000000040000700000000000000"
    vudoku = [
        [7, 2],
        [8, 2],
        [10, 2],
        [15, 0],
        [22, 0],
        [27, 1],
        [28, 3],
        [30, 2],
        [35, 0],
        [36, 1],
        [37, 2],
        [42, 3],
        [49, 2],
        [50, 3],
        [51, 2],
        [52, 1],
        [54, 3],
        [55, 0],
        [57, 3],
        [59, 2],
        [59, 3],
        [70, 0],
        [70, 2],
        [71, 0],
        [73, 1],
        [77, 1],
    ]
    css = CompoundSudokuSolver(grid = test_vudoku_solver, vudoku= vudoku)
    result = css.solveall()
    print(result)

9 6 5 1 4 8 2 7 3 
7 1 2 9 3 6 8 5 4 
3 8 4 2 5 7 6 9 1 
1 2 3 5 6 4 9 8 7 
4 5 6 8 7 9 3 1 2 
8 9 7 3 2 1 4 6 5 
2 7 8 6 1 3 5 4 9 
5 4 9 7 8 2 1 3 6 
6 3 1 4 9 5 7 2 8 

NumConflicts: 0
NumBranches: 0
WallTime: 0.021150000000000002
965148273712936854384257691123564987456879312897321465278613549549782136631495728


![](./assets/figures/Vudoku.png)

-------

# Jigsaw Sudoku 

- 标准数独规则的行、列不重复规则依然适用
- 取消原本3 x 3的宫的概念，而是不规则锯齿状，保证每个锯齿宫依然是9个数字不重复。

---------

- The difference is that instead of having 3x3 rectangular blocks these blocks have irregular shapes

In [14]:
# 测试锯齿数独

if __name__ == "__main__":
    test_jigsaw_solver = "200007000300400000000284600000000000060000030000000000002948000000009008000800001"
    jigsaw = "AAAAABBBCADBBBBCCCADDBBDCCCAADDDDECCFFFDGEEEEFFFGGGEHEFFGGGGEHHFIGIIIEHHIIIIIHHHH"
    css = CompoundSudokuSolver(grid = test_jigsaw_solver, jigsaw = jigsaw)
    result = css.solveall()
    print(result)

2 4 8 6 1 7 3 5 9 
3 8 1 4 9 6 5 2 7 
7 9 5 2 8 4 6 1 3 
9 5 3 7 6 2 1 8 4 
4 6 9 1 7 5 8 3 2 
8 2 7 5 3 1 9 4 6 
1 3 2 9 4 8 7 6 5 
5 1 6 3 2 9 4 7 8 
6 7 4 8 5 3 2 9 1 

NumConflicts: 0
NumBranches: 0
WallTime: 0.013172000000000001
248617359381496527795284613953762184469175832827531946132948765516329478674853291


![](./assets/figures/Jigsaw1.png)

-------

# Greater-than Sudoku (不等式数独)

- 标准数独规则适用
- 某些数字之间存在不等式约束，所填入的格子需要满足这种不等式约束。


-----------


- STD rules applied.
- All numbers must satisfy the Greater-Than Constraints

In [115]:
# 测试不等式数独
if __name__ == "__main__":
    test_greater_than_solver = "000000000000000000000000000000000000000000000000000000000000000000000000000000000"
    greater_than = ""
    css = CompoundSudokuSolver(grid = test_greater_than_solver, greater_than = "<-<->--<<>-->----><---<>>--><>---<---><->>>>--<<-<-<>--<>->----->---><<-<--<<-->----->--<-->><--><----->---<")
    css.solveall()

#  If the number below < the number above, the sign is "<", 
# If the number below > the number above, the sign is ">",
# For each cell, the sign is sequenced from left to right and from top to bottom.
# and iterate cell from left to right and from top to bottom.



3 5 4 7 6 9 2 1 8 
2 9 6 5 1 8 4 3 7 
1 7 8 3 4 2 9 5 6 
7 2 1 9 5 6 3 8 4 
5 4 3 8 2 7 1 6 9 
6 8 9 4 3 1 5 7 2 
4 6 7 1 9 5 8 2 3 
8 3 5 2 7 4 6 9 1 
9 1 2 6 8 3 7 4 5 

NumConflicts: 0
NumBranches: 0
WallTime: 0.019632


![](./assets/figures/GreaterThan1.png)


# 箭头数独（Arrow Sudoku）

- 标准数独规则适用
- 规定每个Arrow起点的数字等于箭头经过的其他数字之和，注意箭头经过的数字可以是重复的

-----------

- STD rules applied
- The number of origin of arrow = sum of rest numbers in the same arrow.


In [15]:
# 测试箭头数独
 

if __name__ == "__main__":
    test_arrow_sudoku_grid = "200000005000100000005020400060003090000040000080500070007060900000007000800000007"
    test_arrow_sudoku_arrow = [
        [(0,4), (0,2), (1,1), (2,0)],
        [(4,8),(2,8),(1,7),(0,6)],
        [(4,0),(6,0),(7,1),(8,2)],
        [(8,4),(8,6),(7,7),(6,8)]
    ]
    css = CompoundSudokuSolver(grid = test_arrow_sudoku_grid, arrow = test_arrow_sudoku_arrow)
    css.solveall()

2 9 3 7 8 4 1 6 5 
6 4 8 1 3 5 7 2 9 
1 7 5 6 2 9 4 8 3 
5 6 4 2 7 3 8 9 1 
7 1 2 9 4 8 5 3 6 
3 8 9 5 1 6 2 7 4 
4 3 7 8 6 1 9 5 2 
9 2 6 4 5 7 3 1 8 
8 5 1 3 9 2 6 4 7 

NumConflicts: 0
NumBranches: 0
WallTime: 0.027005



![](./assets/figures/Arrow%20Sudoku1.png)




- [Ref:Bilibili article](https://www.bilibili.com/read/cv10214658)


-------


# 无马数独(Anti Knight Sudoku)

- 普通数独规则适用
- 国际象棋中马能走到的格子和本格子的数字不能相同

------------------

![](./assets/figures/AntiKnight.png)

--------

Test Case:



# 无缘数独(Anti King Sudoku)

- 普通数独规则适用 
- 国际象棋中King一步能走到的位置数字都不相同(也就是左上、上、右上、左、右、左下、下、右下)都不等于该格。



参考B站大佬的讲解和介绍：[传送门](https://www.bilibili.com/read/cv10214643)

In [100]:
if __name__ == "__main__":
    
    # Test AntiKnight Sudoku
    test_antiknight_grid = "580400027000970000000005030005030000000000680060000300106000000097050000008000000"
    css = CompoundSudokuSolver(grid = test_antiknight_grid, anti_knight=True)
    css.solveall()
    
    # Test Anti-King Sudoku
    test_antiking_grid = "001003070050007009600081000000000745009000300543000000000940003100700050060300200"
    css = CompoundSudokuSolver(grid = test_antiking_grid, anti_king=True)
    css.solveall()

5 8 9 4 6 3 1 2 7 
4 2 3 9 7 1 5 6 8 
6 7 1 8 2 5 4 3 9 
8 1 5 6 3 4 7 9 2 
9 3 2 5 1 7 6 8 4 
7 6 4 2 8 9 3 5 1 
1 4 6 3 9 8 2 7 5 
3 9 7 1 5 2 8 4 6 
2 5 8 7 4 6 9 1 3 

NumConflicts: 0
NumBranches: 0
WallTime: 0.012209000000000001
4 2 1 5 9 3 6 7 8 
3 5 8 6 2 7 4 1 9 
6 9 7 4 8 1 5 3 2 
2 8 6 1 3 9 7 4 5 
7 1 9 8 5 4 3 2 6 
5 4 3 2 7 6 8 9 1 
8 7 5 9 4 2 1 6 3 
1 3 2 7 6 8 9 5 4 
9 6 4 3 1 5 2 8 7 

NumConflicts: 0
NumBranches: 0
WallTime: 0.005952000000000001


![](./assets/figures/Antiknight3.png)

![](./assets/figures/AntiKing.png)

# 三明治数独

- 普通数独规则适用
- 行列上的数字表示这一行/列在1和9之间的数字的和


-----------

- Standard Sudoku rules applied.
- Number of row/ col repressent the sum of numbers between 1 and 9 of this col / row.


In [88]:
# 测试三明治数独

if __name__ == "__main__":

    test_sandwich_grid = "000090064000200100000104007006000000090000806000500000053682010920407005070010020"
    test_sandwich_nums = [
        ["R", 6, 13, 20, 30, 35, 3, 2, 19, 0], 
        ["C", 5,0 ,0, 0, 0, 7, 0, 2, 0]
    ]
    
    css = CompoundSudokuSolver(grid = test_sandwich_grid, sandwich = test_sandwich_nums)
    css.solveall()




7 1 2 3 9 8 5 6 4 
3 4 5 2 7 6 1 9 8 
8 6 9 1 5 4 2 3 7 
5 3 6 8 4 1 9 7 2 
1 9 4 7 2 3 8 5 6 
2 8 7 5 6 9 3 4 1 
4 5 3 6 8 2 7 1 9 
9 2 1 4 3 7 6 8 5 
6 7 8 9 1 5 4 2 3 

NumConflicts: 0
NumBranches: 0
WallTime: 0.017058


![](./assets/figures/Sandwich%20Sudoku1.png)

# Thermometer Sudoku (温度计数独)

- 标准数独规则适用
- 从水银球到顶部，其数字一定是递增的。就像温度计一样。

In [24]:
# 温度计数独

if __name__ == "__main__":
    test_thermo_grid = "785000200000000010000000000000000000000030000000000000000090000004070000100000607"

    test_thermo_cage = [
        [ (0,4), (1,3), (2,2), (3,1) ], 
        [ (4, 8), (3,7), (2,6), (1,5)],
        [(4,5), (3,5), (3,4), (3,3) ], 
        [(4, 3), (5,3), (5,4), (5,5)],
        [ (4,0), (5,1), (6,2), (7,3)],
        [(8,4), (7,5), (6,6), (5,7)]
    ]
    css = CompoundSudokuSolver(grid = test_thermo_grid, thermo = test_thermo_cage)
    css.solveall()
    # Check Completed


7 8 5 3 1 4 2 9 6 
9 4 6 2 5 7 3 1 8 
2 1 3 9 6 8 4 7 5 
6 7 1 5 4 2 8 3 9 
4 9 8 6 3 1 7 5 2 
3 5 2 7 8 9 1 6 4 
8 2 7 1 9 6 5 4 3 
5 6 4 8 7 3 9 2 1 
1 3 9 4 2 5 6 8 7 

NumConflicts: 0
NumBranches: 0
WallTime: 0.011980000000000001


![](./assets/figures/Thermo.png)

In [13]:
# 标准数独

if __name__ == "__main__":
    test_std_solver = "549001738367008001200073040000900005000705460135840070004000307780350006023080000" 
    css = CompoundSudokuSolver(grid=test_std_solver, X = 9, Y = 9, std_rule = True)
    css.solveall()



5 4 9 2 6 1 7 3 8 
3 6 7 4 9 8 5 2 1 
2 1 8 5 7 3 6 4 9 
4 7 6 9 3 2 8 1 5 
8 9 2 7 1 5 4 6 3 
1 3 5 8 4 6 9 7 2 
6 5 4 1 2 9 3 8 7 
7 8 1 3 5 4 2 9 6 
9 2 3 6 8 7 1 5 4 

NumConflicts: 0
NumBranches: 0
WallTime: 0.003268


# 解谜游戏：杀手数独（Killer Sudoku）


- 标准数独规则适用
- 除了标准数独规则外，限制每个虚线框内的数字之和等于标记的数字。


-------

English Version:

Killer Sudoku, Rules:

- Standard Sudoku Applied.
- Sum of numbers in cage equals the marked number.


In [33]:
# 测试杀手数独
if __name__ == "__main__":
    test_killer_solver = "0" * 81
    test_killer_cages = [
        [3, [[0, 0], [0, 1]]],
        [15, [[0, 2], [0, 3], [0, 4]]],
        [22, [[0, 5], [1, 4], [1, 5], [2, 4]]],
        [4, [[0, 6], [1, 6]]],
        [16, [[0, 7], [1, 7]]],
        [15, [[0, 8], [1, 8], [2, 8], [3, 8]]],
        [25, [[1, 0], [1, 1], [2, 0], [2, 1]]],
        [17, [[1, 2], [1, 3]]],
        [9, [[2, 2], [2, 3], [3, 3]]],
        [8, [[2, 5], [3, 5], [4, 5]]],
        [20, [[2, 6], [2, 7], [3, 6]]],
        [6, [[3, 0], [4, 0]]],
        [14, [[3, 1], [3, 2]]],
        [17, [[3, 4], [4, 4], [5, 4]]],
        [17, [[3, 7], [4, 6], [4, 7]]],
        [13, [[4, 1], [4, 2], [5, 1]]],
        [20, [[4, 3], [5, 3], [6, 3]]],
        [12, [[4, 8], [5, 8]]],
        [27, [[5, 0], [6, 0], [7, 0], [8, 0]]],
        [6, [[5, 2], [6, 1], [6, 2]]],
        [20, [[5, 5], [6, 5], [6, 6]]],
        [6, [[5, 6], [5, 7]]],
        [10, [[6, 4], [7, 3], [7, 4], [8, 3]]],
        [14, [[6, 7], [6, 8], [7, 7], [7, 8]]],
        [8, [[7, 1], [8, 1]]],
        [16, [[7, 2], [8, 2]]],
        [15, [[7, 5], [7, 6]]],
        [13, [[8, 4], [8, 5], [8, 6]]],
        [17, [[8, 7], [8, 8]]]
    ]
    
    css = CompoundSudokuSolver(test_killer_solver, killer = test_killer_cages)
    css.solveall()


2 1 5 6 4 7 3 9 8 
3 6 8 9 5 2 1 7 4 
7 9 4 3 8 1 6 5 2 
5 8 6 2 7 4 9 3 1 
1 4 2 5 9 3 8 6 7 
9 7 3 8 1 6 4 2 5 
8 2 1 7 3 9 5 4 6 
6 5 9 4 2 8 7 1 3 
4 3 7 1 6 5 2 8 9 

NumConflicts: 0
NumBranches: 0
WallTime: 0.010554000000000001



![Killer Sudoku](./assets/figures/KillerSudoku.png)

# 解决思维谜题：连续数独（Consecutive Sudoku）

- 标准数独规则适用；
- 有标记的相邻的两个格子数字必须是连续的（差的绝对值是1，因此在实现的时候用 `AllAbsEquality` 函数）

------

English Version:

**Solve Consecutive Sudoku**

- Standard Sudoku Rules applied.
- Two adjacent cage with marker bust be integer-consecutive, (e.g. 1,2 or 2,3; 4,5 or 3,4)


In [26]:
test_consecutive_nums = "000000040000000900000000000003000000000000000001000090000000000000000020000000000"
test_consecutive_constr = "..........C....C.........CC.C.........C..C....C.......C...........C.........C.....C.....C.C..C...........C.....CCC...CC..............CC....C.C.C"
css = CompoundSudokuSolver(grid = test_consecutive_nums, consecutive = test_consecutive_constr )
result = css.solveall()
print(result)

# Consecutive Sudoku(Evil) https://gridpuzzle.com/consecutive-sudoku/21229

8 1 5 7 3 9 2 4 6 
2 7 4 1 8 6 9 5 3 
3 6 9 2 5 4 1 8 7 
7 9 3 5 4 2 8 6 1 
4 2 6 9 1 8 3 7 5 
5 8 1 6 7 3 4 9 2 
6 4 8 3 2 5 7 1 9 
9 3 7 4 6 1 5 2 8 
1 5 2 8 9 7 6 3 4 

NumConflicts: 0
NumBranches: 0
WallTime: 0.014958
815739246274186953369254187793542861426918375581673492648325719937461528152897634


144


-----------

![](./assets/figures/ConsecutiveSudoku.png)

------

# 解决思维谜题：小杀手数独（Petite Killer Sudoku）

- 标准数独规则适用；
- 斜对角线上的数字和等于小数字。注意一共有四个方向的斜对角线，分别用"T","B","L","R" 标记“上(Top)”“下(Bottom)”“左(Left)”“右(Right)”，因此组成了“TL”、“RT”、“BR”、“LB” 四个方向来标记数字。


> Numbers with arrows indicate sum of the numbers in each direction. Direction is shown in each arrow.


In [32]:
# 测试小杀手数独

if __name__ == "__main__":
    test_petite_killer = "005000000000000000000002000000000000000006040000000000000060000000300000000000008"
    test_petite_killer_cages = [ 
        [ "TL", 2, 10, 19, 23, 19, 25, 45, 47], # TL means "Top to Left" (digits from left to right)
        [ "RT", 4, 13, 16, 13, 20, 30, 45, 47], # RT means "Right to Top" (digits from Top to bottom)
        [ "BR", 37, 35, 30, 19, 20, 16, 10, 8], # BR means "Bottom to Right" (digits from left to right)
        [ "LB", 33, 34, 33, 19, 19, 15, 14, 4] # LB means "left to Bottom" (digits from top to bottom)
    ]

    css = CompoundSudokuSolver(grid = test_petite_killer, petite_killer=test_petite_killer_cages )
    css.solveall()

2 9 5 6 3 1 7 8 4 
1 8 3 7 4 9 2 6 5 
6 7 4 8 5 2 1 9 3 
7 2 6 4 9 3 8 5 1 
3 1 8 5 7 6 9 4 2 
5 4 9 2 1 8 3 7 6 
8 3 7 1 6 5 4 2 9 
9 6 2 3 8 4 5 1 7 
4 5 1 9 2 7 6 3 8 

NumConflicts: 0
NumBranches: 1338
WallTime: 0.107007


------


![](./assets/figures/Petite%20Sudoku%201.png)

## 窗口数独 （所有在给定区域内均为1～9不重复的数独）

![](./assets/figures/WindowSudoku.png)

In [4]:
# 测试窗口数独 窗口数独(中级) https://gridpuzzle.com/windoku/1n06k

if __name__ == "__main__":
    test_window_sudoku = "007430000000000690020600000300090006600000009900060003000001080054000000000046200"
    test_window_sudoku_windows = "000000000011102220011102220011102220000000000033304440033304440033304440000000000"

    css = CompoundSudokuSolver(grid = test_window_sudoku, all_nine = test_window_sudoku_windows )
    css.solveall()


8 6 7 4 3 9 1 5 2 
4 3 1 7 2 5 6 9 8 
5 2 9 6 1 8 4 3 7 
3 4 5 8 9 2 7 1 6 
6 1 2 5 7 3 8 4 9 
9 7 8 1 6 4 5 2 3 
7 9 6 2 5 1 3 8 4 
2 5 4 3 8 7 9 6 1 
1 8 3 9 4 6 2 7 5 

NumConflicts: 0
NumBranches: 0
WallTime: 0.010362000000000001


#  XV 数独

- 标记了X的，两个格子的数字和为10，标记了V的，两个格子和为5.

![](./assets/figures/XV_sudoku.png)

In [10]:
if __name__ == "__main__":
    test_XV_sudoku = "000000000000000000000000000000000000000000000000000000000000000090000008000000000"
    test_XV_example = "---------V-VV-V-------------X----X-VX---X--V---XX------X---X----V------X--X----V-X-V-X--X----V---------X-----X----X------X----X--X-V--V---------"
# https://gridpuzzle.com/vx-sudoku/159j0
    css = CompoundSudokuSolver(grid = test_XV_sudoku, XV = test_XV_example )
    css.solveall()

9 3 8 4 2 7 1 5 6 
6 2 5 1 3 8 4 9 7 
7 4 1 9 5 6 2 8 3 
2 1 3 5 6 4 8 7 9 
8 5 9 3 7 1 6 4 2 
4 6 7 2 8 9 3 1 5 
1 8 2 7 9 3 5 6 4 
3 9 4 6 1 5 7 2 8 
5 7 6 8 4 2 9 3 1 

NumConflicts: 0
NumBranches: 0
WallTime: 0.009729


In [6]:
938427156625138497741956283213564879859371642467289315182793564394615728576842931


144


In [8]:
a = "---------V-VV-V-------------X----X-VX---X--V---XX------X---X----V------X--X----V-X-V-X--X----V---------X-----X----X------X----X--X-V--V---------"

for idx, sub_ in enumerate(a):
    if sub_ == "-":
        continue
    sub_a, sub_b = idx // 17 , idx % 17 
    if sub_b <= 7:
        print(f"{sub_}, x[{sub_a}, {sub_b}], x[{sub_a}, {int(sub_b) + 1}]")
    elif sub_b <= 16:
        print(f"{sub_}, x[{sub_a}, {int(sub_b) - 8}], x[{int(sub_a) + 1}, {int(sub_b) - 8}]")
    

V, x[0, 1], x[1, 1]
V, x[0, 3], x[1, 3]
V, x[0, 4], x[1, 4]
V, x[0, 6], x[1, 6]
X, x[1, 3], x[2, 3]
X, x[1, 8], x[2, 8]
V, x[2, 1], x[2, 2]
X, x[2, 2], x[2, 3]
X, x[2, 6], x[2, 7]
V, x[2, 1], x[3, 1]
X, x[2, 5], x[3, 5]
X, x[2, 6], x[3, 6]
X, x[3, 4], x[3, 5]
X, x[3, 0], x[4, 0]
V, x[3, 5], x[4, 5]
X, x[4, 3], x[4, 4]
X, x[4, 6], x[4, 7]
V, x[4, 3], x[5, 3]
X, x[4, 5], x[5, 5]
V, x[4, 7], x[5, 7]
X, x[5, 0], x[5, 1]
X, x[5, 3], x[5, 4]
V, x[5, 0], x[6, 0]
X, x[6, 1], x[6, 2]
X, x[6, 7], x[6, 8]
X, x[6, 4], x[7, 4]
X, x[7, 2], x[7, 3]
X, x[7, 7], x[7, 8]
X, x[7, 2], x[8, 2]
V, x[7, 4], x[8, 4]
V, x[7, 7], x[8, 7]


In [30]:
from __future__ import print_function

import sys
from ortools.constraint_solver import pywrapcp
import numpy as np
from time import time

from random import randint
import numpy as np
from copy import copy,deepcopy


class Skyscraper:

    def __init__(self,N=4,see_list = [],const_list=[]):
        pass
        self.N = N
        #self.state =  np.array([N*[i+1] for i in range(N)])

        self.state = np.array([[0 for _ in range(self.N)] for i in range(self.N)])

        #The way I'm gonna organize this is as a list of lists, where it goes
        #left, right, top, down
        self.see_list = np.array(see_list)

        self.const_list = const_list
        for const in self.const_list:
            ind = const[0]
            val = const[1]
            self.state[ind[0],ind[1]] = val


        self.const_list_indices = [x[0] for x in self.const_list]
        #print(self.const_list)
        #print(self.const_list_indices)

        self.max_FF = 16*(self.N-1) + 8*(self.N)



    def printState(self):
        board = '\n\n'

        #board = board + '  ' + ' '.join([str(x) for x in self.see_list[2]]) + '\n'

        #board = board + '  ' + '__'*(self.N-1) + '_' + ' ' + '\n'

        board = board + '  ' +  '\033[4m' + ' '.join([str(x) for x in self.see_list[2]]) +  '\033[0m' + '\n'

        """for i in range(self.N):

            board = board + str(self.see_list[0][i]) + '|'

            board = board + ' '.join([str(x) for x in self.state[i,:]])

            board = board + '|' + str(self.see_list[1][i])

            board = board + '\n'"""

        for i in range(self.N-1):

            board = board + str(self.see_list[0][i]) + '|'

            board = board + ' '.join([str(x) for x in self.state[i,:]])

            board = board + '|' + str(self.see_list[1][i])

            board = board + '\n'

        board = board + str(self.see_list[0][self.N-1]) + '|'

        board = board + '\033[4m' + ' '.join([str(x) for x in self.state[self.N-1,:]]) + '\033[0m'

        board = board + '|' + str(self.see_list[1][self.N-1])

        board = board + '\n'

        #board = board + '  ' + '‾‾'*(self.N-1) + '‾' + ' ' + '\n'
        board = board + '  ' + ' '.join([str(x) for x in self.see_list[3]]) + '\n'

        print(board)

    def countSeen(self,sel,row_num):
        #will return a tuple of what you see from the left and right if you pass it 'row',
        #up and down if you pass it 'col'

        if sel=='row':
            row = self.state[row_num]
        if sel=='col':
            row = self.state[:,row_num]

        #print(row)
        max_left = max_right = -10000
        seen_left = seen_right = 0

        for i in range(self.N):
            if row[i]>max_left:
                max_left = row[i]
                seen_left += 1
                if row[i]==self.N:
                    break

        #and for from the other direction:
        row = np.flip(row,axis=0)
        #print(row)
        for i in range(self.N):
            if row[i]>max_right:
                max_right = row[i]
                seen_right += 1
                if row[i]==self.N:
                    break

        #print([seen_left,seen_right])
        return([seen_left,seen_right])



    def countOccurrenceErrors(self):

        base_occur = [-1]*self.N

        error_sum = 0

        for i in range(self.N):

            occur_row = copy(base_occur)
            occur_col = copy(base_occur)

            for j in range(self.N):

                occur_row[self.state[i,j]-1] += 1
                occur_col[self.state[j,i]-1] += 1


            #print('there are {} errors in row {}'.format(sum(np.absolute(occur_row)), i))
            #print('there are {} errors in col {}'.format(sum(np.absolute(occur_col)), i))
            error_sum += sum(np.absolute(occur_row)) + sum(np.absolute(occur_col))

        return(error_sum)

    def fitnessFunction(self):

        occur_errors = self.countOccurrenceErrors()

        seen_errors = 0

        for i in range(self.N):
            row_seen = self.countSeen('row',i)
            col_seen = self.countSeen('col',i)

            row_seen_error = sum(np.absolute(row_seen - self.see_list[[0,1],i]))
            col_seen_error = sum(np.absolute(col_seen - self.see_list[[2,3],i]))

            #print('row_seen_error',row_seen_error)
            #print('col_seen_error',col_seen_error)
            seen_errors += row_seen_error + col_seen_error

        return(occur_errors + seen_errors)

    def solFound(self):
        if self.fitnessFunction()<1:
            return(True)
        else:
            return(False)

    def isSameState(self,other):
        return((self.state==other.state).all())

    def mutate(self):
        row = randint(0,self.N-1)
        col = randint(0,self.N-1)
        if [row,col] not in self.const_list_indices:
            self.state[row,col] = randint(1,self.N)


    def mate(self,other):

        new_1 = deepcopy(self)
        new_2 = deepcopy(other)

        r1 = randint(0,self.N-1)
        r2 = randint(r1+1,self.N)

        c1 = randint(0,self.N-1)
        c2 = randint(c1+1,self.N)

        temp_slice = copy(new_1.state[r1:r2,c1:c2])
        new_1.state[r1:r2,c1:c2] = copy(new_2.state[r1:r2,c1:c2])
        new_2.state[r1:r2,c1:c2] = temp_slice

        return((new_1,new_2))





med_88_SS = [[2,3,2,4,4,2,3,1],[1,3,5,3,2,3,2,4],[2,3,2,2,4,6,3,1],[1,2,3,4,3,2,2,4]]
med_88_constlist = [([0,1],1),([1,3],1),([2,1],3),([3,2],3),([4,2],6),([4,3],3),([4,5],5),([5,0],4),([5,4],6),([6,1],2),([6,4],1),([7,4],5)]

hard_88_ss = [[3,2,0,0,0,0,4,0],[3,0,2,4,4,2,0,0],[0,3,0,4,5,4,0,0],[0,3,4,0,2,1,4,2]]
hard_88_constlist = [([1,1],2),([2,1],5),([2,3],3),([4,3],4),([3,7],2),([5,5],1),([6,1],3),([6,7],1),([7,4],3)]

ss_99 = [[1,4,5,2,3,2,3,2,4],[2,2,2,3,4,3,1,3,5],[1,4,3,3,4,3,3,2,2],[4,2,3,2,1,3,2,3,3]]
ss_99_constlist = [([0,3],6),([0,7],2),([1,0],3),([1,8],6),([2,0],5),([2,3],7),
([2,6],2),([3,1],1),([3,5],2),([3,6],8),([4,6],7),([4,7],5),
([5,1],4),([5,3],1),([5,7],7),([5,8],3),([6,1],3),([6,3],5),
([6,4],1),([7,5],3),([7,7],8),([8,5],7)]

see_list = ss_99
const_list = ss_99_constlist
#see_list=[[4,2,2,1],[1,2,2,4],[4,2,2,1],[1,2,2,4]]
#left (t to b), right (t to b), top (l to r), down (l to r)
#s = Skyscraper(4,see_list=[[2,1,3,4],[2,1,3,4],[2,1,3,4],[2,1,3,4]])
size = len(see_list[0])
s = Skyscraper(size,see_list=see_list)
s.printState()


# Creates the solver.
solver = pywrapcp.Solver("simple_example")

#Create the variables we'll solve for
ss_vars = np.array([[solver.IntVar(1, size, "a_{}{}".format(i,j)) for j in range(size)] for i in range(size)])

#CONSTRAINTS

# All rows and columns must be different.
for i in range(len(ss_vars)):
    solver.Add(solver.AllDifferent(ss_vars[i,:].tolist()))
    solver.Add(solver.AllDifferent(ss_vars[:,i].tolist()))

#Make the 'buildings seen' correct.
#I'm going to do it in pairs, because the way it is, I have to reverse stuff.
#'sidepair 0' are going to be the left and right sides, where the var list will have to be switched for the right.
#similarly, sidepair 1 is for the top and bottom.
for entry in range(size):
    #left and right
    sidepair = 0
    left_top = 2*sidepair
    right_bot = 2*sidepair + 1
    #print('adding constraint for left/right sidepair {}, entry {}: {} and {}'.format(sidepair,entry,see_list[left_top][entry],see_list[right_bot][entry]))
    if see_list[left_top][entry]!=0:
        solver.Add(
            (1 + solver.Sum(
                [
                    solver.Min(
                        solver.Max(
                            ss_vars[entry,:j+1].tolist()
                            ) - solver.Max(ss_vars[entry,:j].tolist()),1) for j in range(1,size)]
                ))
            == see_list[left_top][entry]
        )
    if see_list[right_bot][entry]!=0:
        solver.Add((1 + solver.Sum([solver.Min(solver.Max(ss_vars[entry,-(j+1):].tolist()) - solver.Max(ss_vars[entry,-j:].tolist()),1) for j in range(1,size)])) == see_list[right_bot][entry])
    #top and bottom
    sidepair = 1
    left_top = 2*sidepair
    right_bot = 2*sidepair + 1
    #print('adding constraint for left/right sidepair {}, entry {}: {} and {}'.format(sidepair,entry,see_list[left_top][entry],see_list[right_bot][entry]))
    if see_list[left_top][entry]!=0:
        solver.Add((1 + solver.Sum([solver.Min(solver.Max(ss_vars[:j+1,entry].tolist()) - solver.Max(ss_vars[:j,entry].tolist()),1) for j in range(1,size)])) == see_list[left_top][entry])
    if see_list[right_bot][entry]!=0:
        solver.Add((1 + solver.Sum([solver.Min(solver.Max(ss_vars[-(j+1):,entry].tolist()) - solver.Max(ss_vars[-j:,entry].tolist()),1) for j in range(1,size)])) == see_list[right_bot][entry])

#Add constraints for given constants, if there are any
for const in const_list:
    ind = const[0]
    val = const[1]
    solver.Add(ss_vars[ind[0],ind[1]] == val)


#Soluion collector
collector = solver.AllSolutionCollector()
collector.Add(ss_vars.flatten().tolist())

#The "decision builder". I just used the one from:
#https://developers.google.com/optimization/cp/cp_tasks
db = solver.Phase(ss_vars.flatten().tolist(), solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

#Solve it!
time_limit_s = 1
solver.TimeLimit(1000*time_limit_s)
print('\n\nstarting solver with {}s time limit!'.format(time_limit_s))
start_time = time()
solver.Solve(db, [collector])
print('\ndone after {:.2f}s'.format(time()-start_time))

#print solutions
print('\nthis many solutions found:',collector.SolutionCount())
for sol_num in range(collector.SolutionCount()):
    sol = np.array([[collector.Value(sol_num,ss_vars[i,j]) for j in range(size)] for i in range(size)])
    print('\nsolution #{}:\n'.format(sol_num))
    print(sol)




#



  1 4 3 3 4 3 3 2 2
1|0 0 0 0 0 0 0 0 0|2
4|0 0 0 0 0 0 0 0 0|2
5|0 0 0 0 0 0 0 0 0|2
2|0 0 0 0 0 0 0 0 0|3
3|0 0 0 0 0 0 0 0 0|4
2|0 0 0 0 0 0 0 0 0|3
3|0 0 0 0 0 0 0 0 0|1
2|0 0 0 0 0 0 0 0 0|3
4|0 0 0 0 0 0 0 0 0|5
  4 2 3 2 1 3 2 3 3



starting solver with 1s time limit!

done after 0.22s

this many solutions found: 1

solution #0:

[[9 5 7 6 4 1 3 2 8]
 [3 7 8 2 5 4 1 9 6]
 [5 6 3 7 8 9 2 1 4]
 [6 1 9 3 7 2 8 4 5]
 [1 8 4 9 3 6 7 5 2]
 [8 4 6 1 2 5 9 7 3]
 [7 3 2 5 1 8 4 6 9]
 [2 9 1 4 6 3 5 8 7]
 [4 2 5 8 9 7 6 3 1]]


In [4]:
if __name__ == "__main__":
    grid = "000900005000008000300000060000702000001004000074009100600801040402000000700000390"

    even_odd = ".E...OEE.O.O....O...E..OO..OOE......E...O..............OO...E...E.OO.E.......E..."
    
    css = CompoundSudokuSolver(grid = grid, even_odd = even_odd)
    result = css.solveall()
    print(result)

1 4 7 9 6 3 8 2 5 
5 6 9 2 7 8 4 1 3 
3 2 8 1 4 5 7 6 9 
9 3 6 7 1 2 5 8 4 
8 5 1 6 3 4 9 7 2 
2 7 4 5 8 9 1 3 6 
6 9 3 8 5 1 2 4 7 
4 8 2 3 9 7 6 5 1 
7 1 5 4 2 6 3 9 8 

NumConflicts: 0
NumBranches: 0
WallTime: 0.006566000000000001
147963825569278413328145769936712584851634972274589136693851247482397651715426398


![](./assets/figures/EvenOddSudoku.png)

In [8]:
if __name__ == "__main__":
    grid = "0" * 81
    kropki = ".....B.W.....WW....WW..W........W...WW..W..W.B..B...W.W..B....W.W..WB......BW..W............BBW.....WB........WB.BB.....WW.......W..B........B.B"

    css = CompoundSudokuSolver(grid = grid, kropki=kropki)
    result = css.solveall()
    print(result)



5 3 7 9 6 4 8 2 1 
8 1 4 3 2 5 7 6 9 
2 6 9 8 7 1 4 5 3 
9 5 6 4 3 8 2 1 7 
4 2 8 5 1 7 9 3 6 
3 7 1 6 9 2 5 8 4 
6 8 5 1 4 9 3 7 2 
7 4 3 2 8 6 1 9 5 
1 9 2 7 5 3 6 4 8 

NumConflicts: 0
NumBranches: 0
WallTime: 0.022574
537964821814325769269871453956438217428517936371692584685149372743286195192753648


![](./assets/figures/KropkiSudoku.png)

![](./assets/figures/center-dotSudoku.png)

# Reference

> https://github.com/dguzelkokar/Solving-Akari-using-Google-OR-Tools
> 
> src: https://www.kaggle.com/code/pintowar/modeling-a-sudoku-solver-with-or-tools
> 
> src: [Jigsaw](https://cn.gridpuzzle.com/jigsaw-killer-sudoku)
> 
> src: [hakank's site](http://www.hakank.org/google_or_tools/)
>
> [Puzzle Site](https://puzzlemadness.co.uk/slitherlink/large/2023/10/18)
> 
> Sandwich Sudoku: [Github Repo](https://github.com/awkwardbunny/z3-sudoku) in z3 solver
>
> [Solver Site](https://webpbn.com/survey/)
>
> https://github.com/LiorSinai/NonogramSolver/
>
> https://webpbn.com/play.cgi?id=27
>
> https://webpbn.com/survey/
>
> https://rosettacode.org/wiki/Nonogram_solver#Python (`Very Useful!`)
>
> https://github.com/CPMpy/cpmpy/
>
> https://cs.carleton.edu/cs_comps/1516/slither/index.php
